# Shopee Product Classification 

Notebook to experiment over several Neural Networks over the product dataset obtained from Shopee and evaluate results.

The following models are evaluated as part of this notebook:

Baseline 1: CNN <br>
Baseline 2: CNN with augmented layers <br>
Improvement 1: Adding RNN <br>
Improvement 2: Adding attention

## Imports and Config

In [39]:
!pip install scikit-image
!pip install shopee_crawler
!pip install torchvision
!pip install opencv-python

You should consider upgrading via the 'c:\users\shubh\.pyenv\pyenv-win\versions\3.7.9\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\shubh\.pyenv\pyenv-win\versions\3.7.9\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\shubh\.pyenv\pyenv-win\versions\3.7.9\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\shubh\.pyenv\pyenv-win\versions\3.7.9\python.exe -m pip install --upgrade pip' command.


In [1]:
# System
import importlib

# Visualization
import matplotlib.pyplot as plt

import torch
import torch.nn as nn


## Data Load

In [2]:
image_dir = 'data/images'

## Dataset

In [3]:
from model.text import dataset
from model.dataset import all_categories
from scripts.crawler import product_category_and_names

_, product_names = product_category_and_names('data')

data = dataset.DataSet(product_names, max_num_img=300, categories=all_categories)

In [21]:
importlib.reload(dataset)

<module 'model.text.dataset' from '/Users/naomileow/Documents/school/CS5242/project/model/text/dataset.py'>

In [4]:
data.load_all()

100%|██████████| 300/300 [00:00<00:00, 477.41it/s]


## CNN Baseline Model

In [5]:
from model.text import trainer, baseline

batch_size = 32
datastore = trainer.DataStore(data, batch_size)

baseline_model = baseline.BaselineCNN(len(data.categories), datastore.vocab_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(baseline_model.parameters(), lr=5e-4)
mtrainer = trainer.Trainer(baseline_model, optimizer, criterion, batch_size, datastore)

In [28]:
importlib.reload(trainer)
importlib.reload(baseline)

<module 'model.text.baseline' from '/Users/naomileow/Documents/school/CS5242/project/model/text/baseline.py'>

In [6]:
mtrainer.run_train(30)

/opt/homebrew/anaconda3/envs/cs5242-proj/lib/python3.9/site-packages/torch/nn/functional.py:2380: UserWarning: The operator 'aten::_embedding_bag' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:11.)
  ret, _, _, _ = torch.embedding_bag(


epoch   0 |   100 batches loss: 2.9758
[Epoch   0]: Training loss: 2.943993 | Accuracy: 0.130159
[Epoch   0]: Validation loss: 2.919566 | Accuracy: 0.137500 | Within 3: 0.323611
epoch   1 |   100 batches loss: 2.7790
[Epoch   1]: Training loss: 2.766027 | Accuracy: 0.194444
[Epoch   1]: Validation loss: 2.700684 | Accuracy: 0.226389 | Within 3: 0.431944
epoch   2 |   100 batches loss: 2.5695
[Epoch   2]: Training loss: 2.542722 | Accuracy: 0.275794
[Epoch   2]: Validation loss: 2.446675 | Accuracy: 0.290278 | Within 3: 0.550000
epoch   3 |   100 batches loss: 2.2887
[Epoch   3]: Training loss: 2.239562 | Accuracy: 0.375595
[Epoch   3]: Validation loss: 2.183918 | Accuracy: 0.380556 | Within 3: 0.591667
epoch   4 |   100 batches loss: 1.9997
[Epoch   4]: Training loss: 1.961720 | Accuracy: 0.448611
[Epoch   4]: Validation loss: 1.945796 | Accuracy: 0.434722 | Within 3: 0.673611
epoch   5 |   100 batches loss: 1.7502
[Epoch   5]: Training loss: 1.737302 | Accuracy: 0.518056
[Epoch   5]: 

In [7]:
test_loss, test_acc, top_k, incorect_stats = mtrainer.run_test(mtrainer.data.testloader, 3, True)
print(f'Accuracy of the network on the test images: {test_acc*100} %')
print(f'Accuracy within top 3 results: {top_k*100} %')

from collections import Counter

counts = Counter(incorect_stats).most_common(25)
for k, v in counts:
    print(f"({data.categories[k[0]]}, {data.categories[k[1]]}): {v}")

Accuracy of the network on the test images: 60.27777777777777 %
Accuracy within top 3 results: 79.44444444444444 %
(Home-Appliances, Automotive): 15
(Women's-Shoes, Men's-Shoes): 12
(Hobbies-Books, Toys-Kids-Babies): 12
(Beauty-Personal-Care, Home-Living): 12
(Men's-Bags, Women's-Bags): 11
(Home-Living, Toys-Kids-Babies): 10
(Women's-Bags, Men's-Bags): 9
(Kids-Fashion, Toys-Kids-Babies): 9
(Computers-Peripherals, Cameras-Drones): 9
(Computers-Peripherals, Mobile-Gadgets): 9
(Health-Wellness, Toys-Kids-Babies): 7
(Computers-Peripherals, Hobbies-Books): 7
(Home-Appliances, Pet-Food-Supplies): 7
(Mobile-Gadgets, Cameras-Drones): 7
(Beauty-Personal-Care, Toys-Kids-Babies): 7
(Men's-Shoes, Women's-Shoes): 7
(Mobile-Gadgets, Watches): 6
(Beauty-Personal-Care, Automotive): 6
(Travel-Luggage, Men's-Bags): 6
(Home-Appliances, Home-Living): 5
(Health-Wellness, Home-Living): 5
(Women's-Apparel, Jewellery-Accessories): 5
(Hobbies-Books, Cameras-Drones): 5
(Men's-Shoes, Sports-Outdoors): 5
(Home-Li

## CNN Model with skip connections